In [18]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

In [46]:
data, usermap, itemmap, users = load_data2()
data_train, data_val= split_data2(data, 0.2)
data_train_val = data_train + data_val

In [26]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train, verbose=False)
candidate_recommender.fit()

[I 2024-01-02 18:43:47,082] A new study created in memory with name: P3Beta


In [27]:
n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12633,NaN
12634,NaN
12635,NaN


In [28]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff, remove_seen_flag=True)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe

,ItemID
UserID,
0,"[2843, 19966, 15344, 22146, 18210, 8056, 1449,..."
1,"[18653, 22075, 14731, 5564, 49, 16214, 12142, ..."
2,"[5777, 10028, 5136, 1354, 8004, 7070, 18079, 1..."
3,"[227, 49, 21626, 589, 812, 2612, 841, 463, 192..."
4,"[17723, 18426, 14041, 5423, 8448, 18389, 9547,..."
...,...
12633,"[7069, 11493, 2816, 2809, 4959, 2817, 2806, 28..."
12634,"[3951, 11498, 3956, 11497, 17072, 3943, 3932, ..."
12635,"[257, 472, 470, 259, 17223, 248, 1084, 6549, 4..."


In [29]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe

,ItemID
UserID,
0,2843
0,19966
0,15344
0,22146
0,18210
...,...
12637,9618
12637,18479
12637,9590


In [ ]:
#import pandas as pd
#import numpy as np
#from sklearn.model_selection import KFold
#from scipy import sparse as sps
#
## Carica il tuo dataset
## data_train = ...
#
## Definisci il numero di fold desiderato
#k = 5
#
## Inizializza l'oggetto KFold
#kf = KFold(n_splits=k, shuffle=True, random_state=42)
#
#print(kf)
#
#folds = list(kf.split(training_dataframe))
#
#print(len(folds))
#
#cutoff = 50  # Scegli il valore desiderato per il cutoff
#
#for fold, (train_index, val_index) in enumerate(folds):
#    # Dividi il dataset in training e validation
#    train_fold, val_fold = data_val[train_index], data_val[val_index]
#
#    # Valuta il modello su val_fold
#    predictions = []
#
#    for user_id in range(val_fold.shape[0]):
#        recommendations = candidate_recommender.recommend(user_id, cutoff=cutoff, remove_seen_flag=True)
#        predictions.append(recommendations)
#
#
#    # Flatten predictions e val_fold per calcolare le etichette
#    predictions_flat = list(chain.from_iterable(predictions))
#    val_fold_coo = sps.coo_matrix(val_fold)
#    correct_recommendations = pd.DataFrame({"UserID": val_fold_coo.row, "ItemID": val_fold_coo.col})
#    
#    # Calcola le etichette sulla base della corrispondenza tra le previsioni e val_fold
#    labels = [tuple(item) in correct_recommendations.itertuples(index=False, name=None) for item in predictions_flat]
#
#    # Aggiungi le etichette al dataframe
#    training_dataframe.loc[val_index, f"Label_fold_{fold}"] = labels
#
## Aggiungi una colonna "Label" al dataframe che rappresenta l'OR logico delle etichette dei fold
#training_dataframe["Label"] = training_dataframe.filter(like="Label_fold").any(axis=1)
#
## Rimuovi le colonne temporanee dei fold
#training_dataframe.drop(training_dataframe.filter(like="Label_fold"), axis=1, inplace=True)

In [31]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})

correct_recommendations

,UserID,ItemID
0,0,0
1,0,11
2,0,16
3,0,20
4,0,22
...,...,...
95741,12637,11978
95742,12637,12625
95743,12637,13074
95744,12637,14344


In [42]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)

training_dataframe

,UserID,ItemID,Label
0,0,2843,False
1,0,19966,False
2,0,15344,False
3,0,22146,False
4,0,18210,False
...,...,...,...
631895,12637,9618,False
631896,12637,18479,False
631897,12637,9590,False
631898,12637,12293,False


In [ ]:
#for user_id in range(n_users):
#    training_dataframe.loc[user_id, "Label"] = list(reversed(list(range(1, cutoff+1))))
#training_dataframe

In [47]:
topPop = TopPop(data_train_val)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train_val)
p3alpha.fit()

easer_study = optuna.create_study(
    study_name="Easer",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
easer = EASE_R_Recommender(data_train_val)
easer.fit()

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train_val)
itemknncf.fit()


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    "Easer": easer,
    "ItemKNNCF": itemknncf
}

[I 2024-01-02 19:04:25,494] A new study created in memory with name: P3Alpha


P3alphaRecommender: Similarity column 22222 (100.0%), 3021.24 column/sec. Elapsed time 7.36 sec


[I 2024-01-02 19:04:34,327] A new study created in memory with name: Easer


EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 5.82 min


[I 2024-01-02 19:10:57,385] A new study created in memory with name: ItemKNNCF


Similarity column 22222 (100.0%), 7517.95 column/sec. Elapsed time 2.96 sec


In [48]:
import scipy.sparse as sps
from tqdm import tqdm

training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

training_dataframe

100%|██████████| 12638/12638 [00:43<00:00, 287.83it/s]


,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF
0,0,2843,False,7.0,0.007463,0.003122,0.019602
1,0,19966,False,4.0,0.009804,0.002243,0.000000
2,0,15344,False,6.0,0.023902,0.002579,0.009593
3,0,22146,False,1.0,0.001033,0.002192,0.026655
4,0,18210,False,5.0,0.009804,0.002406,0.023710
...,...,...,...,...,...,...,...
631895,12637,9618,False,6.0,0.000000,0.011957,0.066450
631896,12637,18479,False,4.0,0.001826,0.013928,0.158114
631897,12637,9590,False,13.0,0.014328,0.005580,0.017523
631898,12637,12293,False,16.0,0.012555,0.005849,0.000000


In [53]:
item_popularity = np.ediff1d(sps.csc_matrix(data_train_val).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train_val).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
training_dataframe

,UserID,ItemID,Label,TopPop,P3alpha,Easer,ItemKNNCF,item_popularity,user_profile_len
0,0,2843,False,7.0,0.007463,0.003122,0.019602,7,44
1,0,19966,False,4.0,0.009804,0.002243,0.000000,4,44
2,0,15344,False,6.0,0.023902,0.002579,0.009593,6,44
3,0,22146,False,1.0,0.001033,0.002192,0.026655,1,44
4,0,18210,False,5.0,0.009804,0.002406,0.023710,5,44
...,...,...,...,...,...,...,...,...,...
631895,12637,9618,False,6.0,0.000000,0.011957,0.066450,6,71
631896,12637,18479,False,4.0,0.001826,0.013928,0.158114,4,71
631897,12637,9590,False,13.0,0.014328,0.005580,0.017523,13,71
631898,12637,12293,False,16.0,0.012555,0.005849,0.000000,16,71


In [ ]:
training_dataframe = training_dataframe.set_index('ItemID')
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "ItemID"})

In [ ]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [ ]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [ ]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        training_dataframe.drop(columns=["Label"]),
        training_dataframe["Label"],
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=1000)

#Trial 6 finished with value: 0.12485196495596997 and parameters: {'objective': 'rank:ndcg', 'n_estimators': 336, 'learning_rate': 0.023887567768263025, 'reg_alpha': 0.01495634916569455, 'reg_lambda': 0.004873292863879341, 'max_depth': 4, 'max_leaves': 3, 'grow_policy': 'depthwise', 'booster': 'gbtree'}. Best is trial 6 with value: 0.12485196495596997.

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    objective='rank:ndcg', 
    n_estimators=336, 
    learning_rate=0.023887567768263025, 
    reg_alpha=0.01495634916569455, 
    reg_lambda=0.004873292863879341, 
    max_depth=4, 
    max_leaves=3, 
    grow_policy= 'depthwise', 
    booster='gbtree',
    ndcg_exp_gain=False,
    verbosity=0, # 2 if self.verbose else 0,
    enable_categorical=True,
)
XGB_model.fit(
    training_dataframe.drop(columns=["Label"]),
    training_dataframe["Label"],
    group=groups,
    verbose=False
)

In [ ]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)